In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import time
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [3]:
# Data loading and preprocessing
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load all images from the 'JPEGImages' folder
dataset = ImageFolder(root='/content/drive/MyDrive/JPEGImages', transform=data_transforms)


In [4]:
# Calculate weights for each class, adding a small constant 'eps' to avoid division by zero
eps = 1e-6  # Small constant to prevent division by zero
class_counts = np.array([len(np.where(np.array(dataset.targets) == i)[0]) for i in range(len(dataset.classes))])
class_weights = 1. / (class_counts + eps)  # Adding eps to avoid division by zero
weights = class_weights[np.array(dataset.targets)]

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create WeightedRandomSampler for training set
train_targets = [dataset.targets[i] for i in train_dataset.indices]
train_samples_weight = torch.tensor([weights[i] for i in train_dataset.indices])
train_sampler = WeightedRandomSampler(weights=train_samples_weight, num_samples=len(train_samples_weight), replacement=True)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

In [5]:
# Model definition
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 50),  # 50 classes
    nn.LogSoftmax(dim=1)
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 210MB/s]


In [6]:
# Training the model
def train_model():
    model.train()
    for epoch in range(10):  # loop over the dataset multiple times
        start_time = time.time()  # Start time of the epoch
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass with mixed precision
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            # Backward and optimize
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            if i % 20 == 19:  # print every 20 mini-batches
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 20:.4f}')
                running_loss = 0.0

        # Calculate and print the epoch duration
        end_time = time.time()  # End time of the epoch
        epoch_duration = end_time - start_time
        print(f'Epoch {epoch + 1} completed in {epoch_duration:.2f} seconds')


In [7]:
# Evaluate the model
def evaluate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on test images: {100 * correct / total:.2f}%')

In [8]:
# Run training and evaluation
train_model()
evaluate_model()

Epoch 1, Batch 20, Loss: 3.1989
Epoch 1, Batch 40, Loss: 1.7519
Epoch 1, Batch 60, Loss: 1.1045
Epoch 1, Batch 80, Loss: 0.8974
Epoch 1, Batch 100, Loss: 0.7296
Epoch 1, Batch 120, Loss: 0.6488
Epoch 1 completed in 429.38 seconds
Epoch 2, Batch 20, Loss: 0.6312
Epoch 2, Batch 40, Loss: 0.5240
Epoch 2, Batch 60, Loss: 0.5417
Epoch 2, Batch 80, Loss: 0.4854
Epoch 2, Batch 100, Loss: 0.4578
Epoch 2, Batch 120, Loss: 0.4190
Epoch 2 completed in 172.25 seconds
Epoch 3, Batch 20, Loss: 0.4178
Epoch 3, Batch 40, Loss: 0.4066
Epoch 3, Batch 60, Loss: 0.4214
Epoch 3, Batch 80, Loss: 0.3921
Epoch 3, Batch 100, Loss: 0.3573
Epoch 3, Batch 120, Loss: 0.3734
Epoch 3 completed in 81.97 seconds
Epoch 4, Batch 20, Loss: 0.3286
Epoch 4, Batch 40, Loss: 0.3945
Epoch 4, Batch 60, Loss: 0.3355
Epoch 4, Batch 80, Loss: 0.3424
Epoch 4, Batch 100, Loss: 0.3653
Epoch 4, Batch 120, Loss: 0.3490
Epoch 4 completed in 58.87 seconds
Epoch 5, Batch 20, Loss: 0.3014
Epoch 5, Batch 40, Loss: 0.3183
Epoch 5, Batch 60,